In [5]:
using NBInclude
@nbinclude("randommap.ipynb")

randommap (generic function with 1 method)

In [13]:
function convert_maptodcjdist_to_converse(map_to_dedupstr_dcjdist::Dict{Vector{Int}, Tuple{String, Int}})
    dcjdist_to_map = Dict{Int, Vector{Vector{Int}}}()

    for (map, tuple) in map_to_dedupstr_dcjdist
        dcjdist = tuple[2]

        if dcjdist in keys(dcjdist_to_map)
            push!(dcjdist_to_map[dcjdist], map)
        else 
            dcjdist_to_map[dcjdist] = [map]
        end 
    end 

    return dcjdist_to_map
end 

convert_maptodcjdist_to_converse (generic function with 1 method)

In [ ]:
function find_neighbors(max_neighbors, processed_maps, map_to_dedupstr_dcjdist, dcj_dist_to_map, sorted_dists)


end 

In [24]:
# local search heuristic

# total_maps = total number of maps to be created 
# rand_maps = number of maps randomly generated 
# max_neighbors = max number of neighbors explored in each local search
function localsearch(S::String, P::String, total_maps::Int, rand_maps::Int, max_neighbors::Int, mode::String, dupchar_to_unique_chars::OrderedDict{Char, Vector{Char}})
    printstyled("\nSRC " * S * " --> TARGET " * P * "\n", color=:cyan)
    m = 0
    # create arbitrary map for P, a set of random maps S_M, rank maps using estimator algo 
    P_map, P_dedup, map_to_dedupstr_dcjdist = generate_random_maps_and_calc_distances(S, P, rand_maps, dupchar_to_unique_chars, m)
    num_generated_maps = rand_maps
    processed_maps = Set{Vector{Int}}()

    dcj_dist_to_map = convert_maptodcjdist_to_converse(map_to_dedupstr_dcjdist)
    sorted_dists = sort(collect(keys(dcj_dist_to_map)))

    # until 'total_maps' maps are generated
    while length(num_generated_maps) != total_maps
        smallest_dcj_dist = sorted_dists[1]
        maps = dcj_dist_to_map[smallest_dcj_dist]

        # select best not yet explored map 
        map_smallestd = popfirst!(maps)
        if isempty(maps)
            delete!(dcj_dist_to_map, smallest_dcj_dist)
            popfirst!(sorted_dists)
        end 

        # searches up to 'max_neighbors' neighbor maps, adding them to M 
        neighbors = find_neighbors(max_neighbors, processed_maps, map_to_dedupstr_dcjdist, dcj_dist_to_map, sorted_dists)
        
        # track all processed maps (st no repeat processing) 
        num_generated_maps += length(neighbors)
        push!(processed_maps, map_smallestd)
    end
    # return map that returns the shortest DCJ dist 
end 


src = ".a.,aab,bc"  
target = "ba,a,b,ca"

total_maps = 8
rand_maps = 3
max_neighbors = 2

mode = "none"
dupchar_to_unique_chars = OrderedDict{Char, Vector{Char}}()

localsearch(src, target, total_maps, rand_maps, max_neighbors, mode, dupchar_to_unique_chars)


SRC .a.,aab,bc --> TARGET ba,a,b,ca


LoadError: BoundsError: attempt to access 0-element Vector{Int64} at index [1]